note that this notebook can be viewed as a slideshow triggered by: File -> Download as -> Reveal.js slides (.html)

# PySDM tutorial: Intro
### authors: see [github.com/atmos-cloud-sim-uj](https://github.com/atmos-cloud-sim-uj/PySDM/graphs/contributors)
### copyright: Jagiellonian University
    
    
#### tutorial licence: CC-BY
#### tutorial files: [github.com/atmos-cloud-sim-uj/PySDM/tree/master/PySDM_tutorials](https://github.com/atmos-cloud-sim-uj/PySDM/tree/master/PySDM_tutorials)

### *PySDM* : particle-based simulations of atmospheric cloud microphysics

The package features a Pythonic implementation of the Super-Droplet Method (SDM) Monte-Carlo algorithm, hence the name.

PySDM simulates the dynamics of population of particles immersed in moist air. Depending on their size, the simulated particles are referred to as:
- atmospheric aerosol (sub-micrometre particles, nuclei for water condensation),
- cloud droplets (with radius in the 1-100 μm range, negligible fall speed), 
- rain drops (larger precipitating particles formed by collisions).

### The particle-based ansatz in a nutshell:
- the particles are modelled as point-like objects called **super-droplets** with each super-droplet representing a multiplicity of its real-world counterparts with identical properties;

- the **ambient air** is modelled as a continuum medium (fluid flowing with velocity field $\vec{u}$) - perfect gas mixture of dry air and water vapour, characterised by tempreture $\color{orange}{T}$, total and partial pressures $\color{brown}{p}=p_\text{d}+p_\text{v}$ and relative humidity $\color{purple}{RH}=p_\text{v}/p_{\text{sat}}(T)$ (with $\text{d}$ and $\text{v}$ denoting dry-air and water-vapour, respectively, $p_{\text{sat}}(T)$ denoting partial pressure of vapour in phase equilibrium);

- there is **bidirectional coupling** between the ambient air and the particles   
  (in particular: moisture present in the air drives particle growth by condensation, concurrent latent heat release alters the air temperature).

### System state: particles
- attributes of $i$-th ($i = 1, ..., N$) super-droplet:
  - multiplicity: $n^{[i]}$,
  - spatial coordinates: $x$, $y$, $z$, 
  - extensive attributes: particle volume $v^{[i]}$, nucleus volume, ...,
  - intensive attributes: ...
- moist air thermodynamic variables:
  - ...

### Particle dynamics: collisional growth

Collisional growth is commonly modelled by the **Smoluchowski's coagulation equation** (formulated under the assumptions of sufficiently large system and of neglected correlations between numbers of droplets of different sizes):

\begin{eqnarray}
\frac{d}{dt}{c_i} = \frac{1}{2} \sum\limits_{k=1}^{i-1} a_{k, i-k} c_k c_{i-k} - \sum\limits_{k=1}^{\infty} a_{k, i} c_k c_i
\end{eqnarray}

where $a_{i,j}$ is a so-called kernel defining rate of collisions and $c_i$ is the number concentration (in a volume of air) of the particles with volume $i \cdot \Delta v$ where $\Delta v$ is a droplet volume increment.

Note that $c_i$ will correspond to super-droplet multiplicity $n^{[i]}$ (in a unit volume of air) if:
- $v^{[i]} = i \cdot \Delta v$,
- $v^{[i]}$ does not change in time.

In order to relax the above conditions, a congruent stochastic formulation solved with a Monte-Carlo algorithm is used in PySDM.

### Particle dynamics: diffusional growth

Diffusional growth is modelled with the **Fick's** law depicting the proportionality of the vapour flux to the vapour density gradient (ambient $\rho_v$ vs. particle surface $\rho_\circ$), with the flux equated to the ratio of the rate of change of mass $m=\frac{4}{3} \pi r^3 \rho_l$, of a droplet of radius $r$, to the droplet surface $4 \pi r^2$ :

\begin{eqnarray}
\frac{\dot{m}}{4 \pi r^2} =  
  D \frac{\rho_\text{v} - \rho_\circ\left(r, RH, ...\right)}{r}
\end{eqnarray}

Particle growth dynamics resulting from the radius dependence in $\rho_\circ$ have the following phase portrait $\dot{\xi}(\xi)$, where $\xi=r^2$ and $RH$ is the relative humidity:

\begin{eqnarray}
  \dot{\xi} \sim (RH - 1) - a \xi^{-1/2} + b \xi^{-3/2} + \ldots
\end{eqnarray}

![Arabas_and_Shima_2017_Fig1](https://raw.githubusercontent.com/atmos-cloud-sim-uj/PySDM/master/PySDM_tutorials/pics/Arabas_and_Shima_2017_Fig1.svg?sanitize=true)
<center>(Fig. 1 from Arabas & Shima 2017)</center>

### System state: moist air
- particle attributes:
  - ...
- moist air thermodynamic variables: 
  - dry-air density: $\color{red}{\rho_\text{d}}=\rho-\rho_\text{v}$,
  - dry-air potential temperature: $\color{green}{\theta_\text{d}}=T/\Pi(p_d)=T\left(\frac{p_{1000}}{p_\text{d}}\right)^{\frac{R_\text{d}}{c_{p\text{d}}}}$, 
  - water vapour mixing ratio: $\color{blue}{q_\text{v}}=\frac{\rho_\text{v}}{\rho_\text{d}}$,
  
where $p_{1000}=1000 hPa$, $R_\text{d}$ is the gas constant of dry air and $c_{\text{pd}}$ is the constant specific heat of dry air.

### Particles-ambient air coupling

Two simplest frameworks: adiabatic parcel model and a kinematic flow:

$\begin{eqnarray}
\text{Lagrangian (adiabatic):}\\
\color{red}{\partial_t \rho_\text{d}}=\dot{\rho}_\text{d}(t) \\
\color{blue}{\partial_t q_\text{v}} = 0 \\
\color{green}{\partial_t \theta_\text{d}} = 0\\
\text{Eulerian (nondivergent):}\\
\color{red}{\partial_t\rho_\text{d}} = 0 = -\nabla\cdot(\vec{u}\rho_\text{d}) \\
\color{blue}{\partial_t q_\text{v}} = - \rho_\text{d}^{-1} \nabla \cdot (\vec{u} \rho_\text{d} q_\text{v}) \\
\color{green}{\partial_t \theta_\text{d}} = - \rho_\text{d}^{-1} \nabla \cdot (\vec{u} \rho_\text{d} \theta_\text{d})
\end{eqnarray}$
$\qquad$
$\frac{d}{dt} \left[\begin{eqnarray} 
    v^{[i]} \\ 
    \vdots \\
    \rho_\text{d} \\
    \\
    \\
    q_\text{v} \\
    \\
    \theta_\text{d} \\ 
    ~
  \end{eqnarray} \right]$
  $=\left[\begin{eqnarray} 
    \dot{m}(v^{[i]}, \rho_\text{d}, \theta_\text{d}, q_\text{v}) / \rho_l\\
    \vdots \\ 
    \color{red}{\partial_t \rho_\text{d}} \\
    \\
    -\frac{\rho_l}{\rho_\text{d}} \frac{1}{\Delta V} \sum\limits_i n^{[i]} \frac{dv^{[i]}}{dt} + \color{blue}{\partial_t q_\text{v}}
    \\
    -\frac{l_v(T)}{c_p(q_\text{v})} \frac{dq_\text{v}}{dt} \frac{1}{\Pi(\rho_\text{d}, \theta_\text{d})} + \color{green}{\partial_t \theta_\text{d}}
  \end{eqnarray}\right]$

where $l_v(T)$ is the latent heat of evaporation, $c_p(q_\text{v})$ is the specific heat of moist air and $\Delta V$ is the grid cell or parcel volume.

In the Eulerian framework, particle positions evolve due sedimentation with terminal velocity $u_\text{term}$ and due to advection with the flow: $$[\dot{x}^{[i]}, \dot{y}^{[i]}, \dot{z}^{[i]}] = \hat{e}_z u_\text{term}(v^{[i]}) + \vec{u}$$